![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Inicio

In [1]:
# Importar QuantBook
from QuantConnect.Research import QuantBook

# Importar otros módulos necesarios
from QuantConnect import *
from QuantConnect.Algorithm import QCAlgorithm
from QuantConnect.Data.Market import TradeBar, QuoteBar
from datetime import datetime

# Iniciar QuantBook
qb = QuantBook()

In [2]:
# Set time zone to mexico
qb.SetTimeZone("America/Mexico_City")

# Datasets

Hay diferentes formas de obtener datos con quantconnect:

- **Periodos de tiempo**

    Puedes obtener datos a partir de una ventana temporal, o por periodos específicos de tiempo. Los objetos datetime se basan en el timezone de la libreta.
- **Formatos de retorno**

    Cada clase de activo tiene formatos diferentes.

    También, si no se especifica qué tipo de objeto quieres, se obtendrá un objeto *Slice*. Para obtener múltiplos objetos, se usa *symbol* (útil para manejar los datos de un activo de forma independiente). **Se debe de específicar al momento de suscribirse si se desean symbols**.

    Se pueden obtener Dataframes de forma directa. Sin embargo, si sólo se necesitan Tradebars o Quotebars, se deberían de pedir directamente para evitar gastar recursos computacionales creando el dataframe.

    - Tradebar:
        - Representa las transacciones de un activo en un periodo de tiempo específico
        
        - Contiene información de apretura, cierre, máximo, mínimo y volumen.

    - QuoteBar
        - Representa un resumen de las cotizaciones de compra y venta.
        - Contiene información sobre precios de oferta y demanda
        - Tienen más información.

    - Dataframes
        - Tienen todo lo anterior. Facilita el manejo de los datos, a expensas de costo computacional.
    
    - Tick bars
        - Información de cada transacción individual. Datos con alta granularidad, detallando cada cambio en el precio y volumen de un activo.
        

    

In [3]:
# Subscribe to ES futures data
ES_symbol = qb.AddFuture("ES").Symbol

# Define the start and end date for the historical data request
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 31)


# Request dataframe with historical data for the ES futures contract with minute resolution
es_data = qb.History([ES_symbol], start_date, end_date, Resolution.Minute)

# Request trade bars 
es_trade_bars = qb.History[TradeBar](ES_symbol, start_date, end_date, Resolution.Minute)


# Request quote bars 
es_quote_bars = qb.History[QuoteBar](ES_symbol, start_date, end_date, Resolution.Minute)



In [8]:
# View the tradebar structure, uncomment to see the data
#for trade_bar in es_trade_bars:
    #print(trade_bar)

/ES: O: 4091.6048 H: 4092.1319 L: 4086.8606 C: 4088.4420 V: 14760
/ES: O: 4088.1784 H: 4088.4420 L: 4082.6435 C: 4086.8606 V: 9564
/ES: O: 4086.8606 H: 4091.8683 L: 4086.8606 C: 4088.9691 V: 6461
/ES: O: 4088.9691 H: 4098.4575 L: 4088.9691 C: 4098.4575 V: 7856
/ES: O: 4098.4575 H: 4099.2482 L: 4091.8683 C: 4092.6590 V: 7229
/ES: O: 4092.6590 H: 4097.9303 L: 4092.3955 C: 4095.0311 V: 6616
/ES: O: 4094.7675 H: 4095.8218 L: 4090.5505 C: 4092.6590 V: 4268
/ES: O: 4092.6590 H: 4096.3489 L: 4091.0776 C: 4094.7675 V: 4500
/ES: O: 4094.5040 H: 4096.6125 L: 4093.4497 C: 4095.5582 V: 3514
/ES: O: 4095.5582 H: 4102.6745 L: 4095.2947 C: 4101.8838 V: 6259
/ES: O: 4102.1474 H: 4106.3645 L: 4099.7753 C: 4105.5738 V: 7242
/ES: O: 4105.8373 H: 4110.8451 L: 4105.8373 C: 4109.7908 V: 7892
/ES: O: 4109.7908 H: 4113.2172 L: 4109.5273 C: 4110.3180 V: 6811
/ES: O: 4110.3180 H: 4111.3722 L: 4107.6823 C: 4108.2094 V: 4788
/ES: O: 4108.2094 H: 4109.7908 L: 4099.5117 C: 4100.8296 V: 7656
/ES: O: 4101.0931 H: 410

Si sólo se necesitan los datos, se pueden obtener directamente los objetos. También se pueden obtener como dataframes, lo cuál facilita la visualización y su manejo. 

In [5]:
# Request Gold futures tradebar data by timedelta

gold_symbol = qb.AddFuture("GC").Symbol
gold_trade_bars = qb.History[TradeBar](gold_symbol, timedelta(days=30), Resolution.Minute)

In [13]:
# Request a dataframe of gold_trade_bars
gold_trade_bars_df = qb.history(TradeBar, [gold_symbol], timedelta(days=30))


# Consolidación de datos

De acuerdo a Lopez de Prado en Advances in Financial Machine learning, hay mejores formas de obtener muestras (barras) que por resolución temporal. Se pueden crear barras de volumen (número de transacciones), o incluso de dolares (Cantidad de dolares en transacción en el mercado). Este tipo de muestreo muestra mejores propiedades estadísticas como una distribución más cercana a la IID.



In [1]:


# Agregar la suscripción al contrato de futuros del S&P 500 E-Mini
future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Minute, 
                      dataNormalizationMode=DataNormalizationMode.BackwardsRatio,
                      dataMappingMode=DataMappingMode.LastTradingDay,
                      contractDepthOffset=0)

# Establecer un filtro para el contrato (opcional dependiendo del análisis)
future.SetFilter(0, 182)  # Filtrar contratos con hasta 6 meses hasta la expiración

# Establecer el rango de fechas para la obtención de datos históricos
start_date = datetime(2023, 3, 1)
end_date = datetime(2024, 1, 1)

# Obtener datos históricos para el rango de fechas especificado
# Se utiliza GetFutureHistory para obtener datos históricos del contrato continuo
future_history = qb.GetFutureHistory(future.Symbol, start_date, end_date, 
                                     Resolution.Minute, fillForward=True, 
                                     extendedMarketHours=False)

# Convertir FutureHistory a DataFrame para análisis
future_history_df = future_history.GetAllData()

# Imprimir o analizar el DataFrame
print(future_history_df.head())


In [2]:
import matplotlib.pyplot as plt
import pandas as pd

# Asegurarse de que el índice esté en formato datetime
future_history_df.index = pd.to_datetime(future_history_df.index.get_level_values(2))

# Suponiendo que queremos graficar los datos de todos los contratos juntos sin seleccionar uno específico
# Podemos agrupar por la fecha y calcular el promedio de los precios de cierre para simplificar
# Esto es solo un ejemplo, en la práctica, querrías seleccionar un contrato específico o ajustar el análisis a tus necesidades
daily_close_avg = future_history_df.groupby(future_history_df.index).mean()['close']

# Graficar los precios de cierre promedio diarios
plt.figure(figsize=(15, 8))
plt.plot(daily_close_avg.index, daily_close_avg, label='Precio de Cierre Promedio Diario')

plt.title('Precios de Cierre Promedio Diario del Contrato de Futuros del S&P 500 E-Mini')
plt.xlabel('Fecha')
plt.ylabel('Precio de Cierre Promedio')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [3]:
# Asumiendo que 'future_history_df' es el DataFrame que contiene los datos históricos obtenidos

# Imprimir los nombres de las columnas para identificar las columnas de interés
print("Nombres de columnas:", future_history_df.columns)

# Imprimir los niveles de índice para entender la estructura del índice multinivel
print("Niveles de índice:", future_history_df.index.names)

# Identificar y mostrar el primer registro para entender la estructura de los datos
print("Primer registro del DataFrame:", future_history_df.iloc[0])


In [4]:
# Crear un nuevo DataFrame con las columnas de interés: 'close' y 'volume'
# Utilizamos el índice del DataFrame original para el tiempo
working_df = future_history_df[['close', 'volume']].copy()

# Renombrar el índice para reflejar que representa el tiempo de cierre
working_df.index.rename('Close Time', inplace=True)

# Agregar una columna de símbolo
# Nota: Como no tenemos un identificador de símbolo explícito en los datos proporcionados,
# asumiremos un símbolo genérico o necesitamos ajustar esto según cómo se identifique el símbolo en tu contexto específico
working_df['Symbol'] = 'ES'  # Ajustar según sea necesario

# Reordenar las columnas para poner 'Symbol' primero
working_df = working_df[['Symbol', 'close', 'volume']]

# Mostrar las primeras filas del nuevo DataFrame
print(working_df.head())


## Consolidar barras a 30 minutos para compararlas con Volume y Dollar Bars

In [5]:
# Restablecer el índice para dejar 'time' como la única columna de índice
future_history_df.reset_index(inplace=True)

# Convertir la columna 'time' a datetime si aún no lo es
future_history_df['time'] = pd.to_datetime(future_history_df['time'])

# Establecer 'time' como el nuevo índice del DataFrame
future_history_df.set_index('time', inplace=True)

# Ahora aplicar resample para consolidar las barras a 30 minutos
# 'close' tomará el último valor de cada período de 30 minutos y 'volume' la suma de los volúmenes
consolidado_df = future_history_df.resample('30T').agg({
    'close': 'last',
    'volume': 'sum'
})

# Calcular el cambio de precio ("price change")
consolidado_df['price_change'] = consolidado_df['close'].diff()

# Calcular el retorno como el cambio de precio dividido por el precio de cierre de la barra anterior
consolidado_df['return'] = consolidado_df['price_change'] / consolidado_df['close'].shift()

# Verificar el DataFrame con las nuevas columnas añadidas
print(consolidado_df.head())


## Creación de Volume Bars

In [6]:
import pandas as pd

# Inicializar variables
volumen_acumulado = 0
precio_cierre_anterior = None  # Inicializar como None
volume_bars = []

# Iterar sobre el DataFrame
for index, row in working_df.iterrows():
    volumen_acumulado += row['volume']
    
    # Verificar si se alcanza el umbral
    if volumen_acumulado >= 17590:
        if precio_cierre_anterior is not None:
            # Calcular el retorno desde la última barra si no es el primer registro
            retorno = (row['close'] - precio_cierre_anterior) / precio_cierre_anterior
            # Calcular el cambio de precio desde la última barra
            price_change = row['close'] - precio_cierre_anterior
        else:
            # Para el primer registro, establecer retorno y price_change como NaN o 0 según la preferencia
            retorno = None
            price_change = None
        
        # Registrar la barra
        volume_bars.append({
            'Fecha': index,
            'PrecioCierre': row['close'],
            'Volumen': volumen_acumulado,
            'PriceChange': price_change,
            'Retorno': retorno
        })
        
        # Resetear el volumen acumulado y actualizar el precio de cierre anterior
        volumen_acumulado = 0
        precio_cierre_anterior = row['close']

# Crear un nuevo DataFrame con las volume bars
df_volume_bars = pd.DataFrame(volume_bars)

# Mostrar el nuevo DataFrame
print(df_volume_bars.head())


## Creación de Dollar Bars

In [7]:
# Pseudocode to create dollar bars based on the given average value and volume per bar.

# Define the dollar value per bar using the given average value and volume.
average_value = 4427
volume_per_bar = 16500
dollar_value_per_bar = average_value * volume_per_bar

# Initialize the accumulator for the dollar value and the list to store dollar bars.
accumulated_dollar_value = 0
dollar_bars = []

# We will also keep track of the start time of each bar to include in the dollar bars.
start_time = None

# Pseudocode for iterating over the DataFrame and creating dollar bars.
for index, row in future_history_df.iterrows():
    # If we're starting a new bar, save the current time as the start time for the bar.
    if accumulated_dollar_value == 0:
        start_time = index
        
    # Calculate the dollar value for the current row.
    current_dollar_value = row['close'] * row['volume']
    
    # Add the current dollar value to the accumulator.
    accumulated_dollar_value += current_dollar_value
    
    # If the accumulated dollar value reaches the threshold, create a new dollar bar.
    if accumulated_dollar_value >= dollar_value_per_bar:
        # Calculate the return and price change only if there is a previous bar.
        if len(dollar_bars) > 0:
            previous_close = dollar_bars[-1]['close']
            price_change = row['close'] - previous_close
            # Avoid division by zero.
            if previous_close != 0:
                return_pct = price_change / previous_close
            else:
                return_pct = 0
        else:
            # For the first bar, there is no previous close, so set to None or zero.
            price_change = None
            return_pct = None
        
        # Append the new bar to the list of dollar bars.
        dollar_bars.append({
            'start_time': start_time,
            'end_time': index,
            'close': row['close'],
            'volume': accumulated_dollar_value / row['close'],  # Estimate the volume.
            'price_change': price_change,
            'return': return_pct
        })
        
        # Reset the accumulator for the next bar.
        accumulated_dollar_value = 0

# Convert the list of dollar bars to a DataFrame.
dollar_bars_df = pd.DataFrame


In [10]:
dollar_bars_df.

## Análisis descriptivo de cada tipo de barra

In [11]:
# Resumen estadístico descriptivo de los Volume bars
resumen_volume_bars = df_volume_bars.describe()

# Imprimir los resúmenes
print("Resumen estadístico descriptivo de los Volume bars:")
print(resumen_volume_bars)

In [12]:
# Resumen estadístico descriptivo de las barras de 30 minutos
resumen_consolidado_df = consolidado_df.describe()
print("\nResumen estadístico descriptivo de las barras de 30 minutos:")
print(resumen_consolidado_df)

## Visualizaciónes

In [13]:
# Convertir volume_bars a DataFrame si es necesario
if isinstance(volume_bars, list):
    volume_bars_df = pd.DataFrame(volume_bars)
else:
    volume_bars_df = volume_bars  # Asumiendo que volume_bars ya es un DataFrame


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarte de que volume_bars sea un DataFrame
if isinstance(volume_bars, list):
    volume_bars_df = pd.DataFrame(volume_bars)
else:
    volume_bars_df = volume_bars  # Si ya es un DataFrame, úsalo directamente

# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Crear figuras para los boxplots
plt.figure(figsize=(18, 6))

# Boxplot de volumen para volume_bars_df y consolidado_df
plt.subplot(1, 3, 1)
sns.boxplot(data=[volume_bars_df['Volumen'], consolidado_df['volume']], palette="coolwarm")
plt.xticks([0, 1], ['Volume Bars', 'Consolidado'])
plt.title('Boxplot de Volumen')

# Boxplot de retornos para volume_bars_df y consolidado_df
plt.subplot(1, 3, 2)
sns.boxplot(data=[volume_bars_df['Retorno'], consolidado_df['return']], palette="coolwarm")
plt.xticks([0, 1], ['Volume Bars', 'Consolidado'])
plt.title('Boxplot de Retornos')




In [15]:
# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Crear figura para las distribuciones de retornos
plt.figure(figsize=(12, 6))

# Graficar la distribución de retornos de volume_bars_df
sns.histplot(volume_bars_df['Retorno'], kde=True, color="blue", label='Retornos de Volume Bars', stat="density", bins=30)

# Graficar la distribución de retornos de consolidado_df
sns.histplot(consolidado_df['return'], kde=True, color="red", label='Retornos de Consolidado', stat="density", bins=30)

# Configurar título y leyenda
plt.title('Distribución de Retornos')
plt.xlabel('Retorno')
plt.ylabel('Densidad')
plt.legend()

plt.tight_layout()
plt.show()



In [16]:
# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Crear figura para las distribuciones de retornos
plt.figure(figsize=(12, 6))

# Graficar la distribución de retornos de volume_bars_df
sns.histplot(volume_bars_df['Retorno'], kde=True, color="blue", label='Retornos de Volume Bars', stat="density", bins=30)

# Graficar la distribución de retornos de consolidado_df
sns.histplot(consolidado_df['return'], kde=True, color="red", label='Retornos de Consolidado', stat="density", bins=30)

# Configurar título y leyenda
plt.title('Distribución de Retornos')
plt.xlabel('Retorno')
plt.ylabel('Densidad')
plt.legend()

plt.tight_layout()
plt.show()



In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Graficar la distribución de retornos para consolidado_df y volume_bars_df
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.histplot(consolidado_df['return'].dropna(), kde=True, color="blue", label='Consolidado')
plt.title('Distribución de Retornos de Consolidado')
plt.legend()

plt.subplot(1, 2, 2)
sns.histplot(volume_bars_df['Retorno'].dropna(), kde=True, color="red", label='Volume Bars')
plt.title('Distribución de Retornos de Volume Bars')
plt.legend()

plt.tight_layout()
plt.show()


In [18]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt

# Configurar el espacio de la figura y los ejes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Autocorrelación para consolidado_df excluyendo el lag 0
plot_acf(consolidado_df['return'].dropna(), lags=range(1, 21), alpha=0.05, ax=ax1)
ax1.set_title('Autocorrelación de Retornos de Consolidado')

# Autocorrelación para volume_bars_df excluyendo el lag 0
plot_acf(volume_bars_df['Retorno'].dropna(), lags=range(1, 21), alpha=0.05, ax=ax2)
ax2.set_title('Autocorrelación de Retornos de Volume Bars')

plt.tight_layout()
plt.show()




In [19]:
# Graficar los retornos para inspeccionar visualmente la heteroscedasticidad
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(volume_bars_df['Retorno'], marker='o', linestyle='', markersize=2)
plt.title('Retornos de Volume Bars')
plt.xlabel('Tiempo')
plt.ylabel('Retorno')

plt.subplot(1, 2, 2)
plt.plot(consolidado_df['return'], marker='o', linestyle='', markersize=2)
plt.title('Retornos de Consolidado')
plt.xlabel('Tiempo')
plt.ylabel('Retorno')

plt.tight_layout()
plt.show()


# Misc

In [20]:
import pandas as pd
import numpy as np

# Asegurarse de que el índice está en formato datetime si no lo está
working_df.index = pd.to_datetime(working_df.index)

# Calcular el volumen total diario
volumen_total_diario = working_df['volume'].resample('D').sum()

# Dividir el volumen total diario por 50 para obtener el tamaño objetivo de cada "volume bar"
tamaño_volume_bar = volumen_total_diario / 50

# Calcular el promedio global diario de estos tamaños de "volume bars"
promedio_global_diario = tamaño_volume_bar.mean()

# Mostrar el promedio global diario
print(f"La idea es construir aproximadamente 50 barras diarias")
print(f"Promedio global diario de volumen para 50 'volume bars' diarios: {promedio_global_diario}")

